In [ ]:
import torch
import torchvision
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score,recall_score, confusion_matrix
import random


In [ ]:
# Train and Test Datasets
batch_size_train = 64
batch_size_test = 1000

random_seed = 1
torch.manual_seed(random_seed)

In [ ]:
#Get MNIST dataset
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [ ]:
# Convert training data
train_data = enumerate(train_loader)
batch_idx, (train_data_init, train_targets_init) = next(train_data)

train_data = torch.zeros((batch_size_train,28*28))
train_targets = torch.zeros((batch_size_train,10))
for i in range(batch_size_train):
    train_data[i] = torch.flatten(train_data_init[i][0])
    tmp = torch.zeros((10))
    tmp[train_targets_init[i]] = 1.0
    train_targets[i] = tmp

In [ ]:
# Convert Testing data
test_data = enumerate(test_loader)
batch_idx, (test_data_init, test_targets) = next(test_data)
test_data = torch.zeros((batch_size_test,28*28))

for i in range(batch_size_test):
    test_data[i] = torch.flatten(test_data_init[i][0])



In [ ]:
# MNIST Classifier

class MNISTclassifier(nn.Module):
    def __init__(self, input_size: int):
        super(MNISTclassifier, self).__init__()
        self.input_layer = nn.Linear(input_size, 512, bias=True)
        self.second_layer = nn.Linear(512, 64, bias=True)
        self.third_layer = nn.Linear(64, 10, bias=True)
        self.relu = nn.LeakyReLU(0.2)
        self.soft = nn.Softmax(dim=0)

    def forward(self, x):
        out = self.input_layer(x)
        out = self.relu(out)
        out = self.second_layer(out)
        out = self.relu(out)
        out = self.third_layer(out)
        out = self.soft(out)
        return out

In [ ]:
# Define Hyperparameters
learning_rate = 0.01
number_of_iterations = 10000


MNIST = MNISTclassifier(28*28)
criterian = nn.MSELoss()
optimizer = torch.optim.Adadelta(MNIST.parameters(), lr=learning_rate)

for epoch in range(number_of_iterations):
    order = [i for i in range(batch_size_train)]
    random.shuffle(order)
    for i in order:
        nn_out = MNIST(torch.flatten(train_data_init[i][0]))
        loss = criterian(nn_out, train_targets[i].float())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    if epoch%(number_of_iterations/10) == 0:
        predict_out = torch.zeros((batch_size_train))
        for i in range(batch_size_train):
          tmp_list = MNIST(torch.flatten(train_data_init[i][0])).tolist()
          predict_out[i] = tmp_list.index(max(tmp_list))

        predict_y=torch.round(predict_out,decimals=0).int()

        print(f'Epoch: {epoch} Loss: {loss} Prediction accuracy: {accuracy_score(train_targets_init.data, predict_y.data)}')


Epoch: 0 Loss: 0.09007976949214935 Prediction accuracy: 0.125
Epoch: 1000 Loss: 0.0004847997333854437 Prediction accuracy: 1.0
Epoch: 2000 Loss: 3.6593861295841634e-05 Prediction accuracy: 1.0
Epoch: 3000 Loss: 8.225408237194642e-05 Prediction accuracy: 1.0
Epoch: 4000 Loss: 3.867715349770151e-06 Prediction accuracy: 1.0
Epoch: 5000 Loss: 4.8067009629448876e-05 Prediction accuracy: 1.0
Epoch: 6000 Loss: 9.29932593862759e-06 Prediction accuracy: 1.0
Epoch: 7000 Loss: 2.7762269382947125e-05 Prediction accuracy: 1.0
Epoch: 8000 Loss: 1.242377294374819e-07 Prediction accuracy: 1.0
Epoch: 9000 Loss: 8.084980436251499e-07 Prediction accuracy: 1.0


In [ ]:
#Testing
predict_out = torch.zeros((batch_size_test))

for i in range(batch_size_test):
    tmp_list = MNIST(torch.flatten(test_data_init[i][0])).tolist()
    predict_out[i] = tmp_list.index(max(tmp_list))




In [ ]:
print('------------Accuracy----------------------')
predict_y=torch.round(predict_out,decimals=0).int()

print('prediction accuracy',accuracy_score(test_targets.data, predict_y.data))

print('macro precision',precision_score(test_targets.data, predict_y.data, average='macro'))
print('micro precision',precision_score(test_targets.data, predict_y.data, average='micro'))

print('macro recall',recall_score(test_targets.data, predict_y.data, average='macro'))
print('micro recall',recall_score(test_targets.data, predict_y.data, average='micro'))

------------Accuracy----------------------
prediction accuracy 0.707
macro precision 0.7030843402571438
micro precision 0.707
macro recall 0.7009005336729833
micro recall 0.707


In [ ]:
# Export Classifier model
torch.save(MNIST, 'C.pkl')

In [ ]:
generator = torch.load('G.pkl')